In [77]:
from mesa import Agent, Model 
from mesa.time import RandomActivation
from mesa.space import MultiGrid 
#from mesa.datacollection import DataCollector 
from mesa.visualization.modules import CanvasGrid, ChartModule
from mesa.experimental import JupyterViz

import solara
from matplotlib.figure import Figure
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

import random
import time

In [78]:
class RobotAgent(Agent):
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.steps = 0

    def move(self):
        possible_moves = self.model.grid.get_neighborhood(
            self.pos, 
            moore=True, 
            include_center=False)
        new_position = self.random.choice(possible_moves)
        self.model.grid.move_agent(self, new_position)

    def step(self):
        self.steps += 1
        cell_contents = self.model.grid.get_cell_list_contents([self.pos])
        dirty_cell = next((agent for agent in cell_contents if isinstance(agent, DirtyCellAgent)), None)

        if dirty_cell:
            self.model.grid.remove_agent(dirty_cell)
            self.model.dirty_cells -= 1
        else:
            self.move()

class DirtyCellAgent(Agent):
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)

    def step(self):
        pass

In [79]:
class CleanerModel(Model):
    def __init__(self, width, height, num_agents, dirt_percentage, max_steps):
        self.num_agents = num_agents
        self.schedule = RandomActivation(self)
        self.grid = MultiGrid(width, height, False)
        self.max_steps = max_steps
        self.steps = 0
        self.running = True

        num_dirty_cells = int((dirt_percentage / 100) * (width * height))
        self.dirty_cells = num_dirty_cells

        for i in range(self.num_agents):
            agent = RobotAgent(i, self)
            self.schedule.add(agent)
            x = 1
            y = 1
            self.grid.place_agent(agent, (x, y))

        for i in range(num_dirty_cells):
            dirty_cell = DirtyCellAgent(self.num_agents + i, self)
            self.schedule.add(dirty_cell)
            x, y = random.choice(list(self.grid.empties))
            self.grid.place_agent(dirty_cell, (x, y))

    def step(self):
        self.schedule.step()
        self.steps += 1
        if self.steps >= self.max_steps or self.dirty_cells == 0:
            self.running = False

In [90]:
def make_agent_counts_chart(model):
    fig = Figure()
    ax = fig.subplots()
    num_cleaners = sum(1 for agent in model.schedule.agents if isinstance(agent, RobotAgent))
    dirty_cells = [model.dirty_cells] * model.schedule.steps
    data = {'Steps': model.schedule.steps, 'Number of Cleaners': [num_cleaners] * model.schedule.steps, 'Dirty Cells': dirty_cells}
    df = pd.DataFrame(data)
    sns.lineplot(x='Steps', y='Number of Cleaners', data=df, label='Number of Cleaners', ax=ax, marker= 'o')
    sns.lineplot(x='Steps', y='Dirty Cells', data=df, label='Dirty Cells', ax=ax, marker= 'o')
    ax.set_xlabel('Steps')
    ax.set_ylabel('Count')
    ax.legend()
    return solara.FigureMatplotlib(fig)

def make_cleaning_efficiency_chart(model):
    fig = Figure()
    ax = fig.subplots()
    total_cells = model.grid.width * model.grid.height
    cleaned_cells = total_cells - model.dirty_cells
    cleaning_efficiency = cleaned_cells / total_cells
    data = {'Steps': model.schedule.steps, 'Cleaning Efficiency': [cleaning_efficiency] * model.schedule.steps}
    df = pd.DataFrame(data)
    sns.lineplot(x='Steps', y='Cleaning Efficiency', data=df, label='Cleaning Efficiency', ax=ax, marker='o')
    ax.set_xlabel('Steps')
    ax.set_ylabel('Cleaning Efficiency')
    ax.legend()
    return solara.FigureMatplotlib(fig)

In [91]:
def my_agent_portrayal(agent):
    if isinstance(agent, DirtyCellAgent):
        return {
            "color": "tab:red",
            "size": 50
        }
    elif isinstance(agent, RobotAgent):
        return {
            "color": "tab:blue",
            "size": 50 
        } 

model_params = {
    "width": 10,
    "height": 10,
    "num_agents": {
        "type": "SliderInt",
        "value": 50,
        "label": "Number of Cleaner Robots:",
        "min": 10,
        "max": 100,
        "step": 1
    },
    "dirt_percentage": {
        "type": "SliderInt",
        "value": 20,
        "label": "Percentage of dirty cells:",
        "min": 10,
        "max": 100,
        "step": 1
    },
    "max_steps": {
        "type": "SliderInt",
        "value": 100,
        "label": "Maximum number of steps:",
        "min": 10,
        "max": 1000,
        "step": 1
    }
}

page = JupyterViz(
    CleanerModel,
    model_params,
    measures = [make_cleaning_efficiency_chart,
               make_agent_counts_chart],
    name="Cleaner Model",
    agent_portrayal = my_agent_portrayal
)

page

Cannot show ipywidgets in text